# ICD Project

## Importing libraries

In [203]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from gensim.utils import simple_preprocess

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/marta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/marta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/marta/nltk_data...


True

## Load Dataset

In [204]:
data = pd.read_csv('~/Downloads/icd_marta_ana_scopus.csv')
data

,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,ISBN,CODEN,PubMed ID,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Open Access,Source,EID
0,Casillo M.; Conte D.; Lombardi M.; Santaniello...,"Casillo, Mario (23026997200); Conte, Dajana (2...",23026997200; 22834021600; 57189595215; 5719787...,A Content-Based Recommender System for Hidden ...,2022,Lecture Notes in Networks and Systems,217,NaN,NaN,97,...,978-981162101-7,NaN,NaN,English,Lect. Notes Networks Syst.,Conference paper,Final,NaN,Scopus,2-s2.0-85118993536
1,Rani C.J.; Devarakonda N.,"Rani, Challapalli Jhansi (57870952400); Devara...",57870952400; 54999915100,An effectual classical dance pose estimation a...,2022,Microprocessors and Microsystems,95,NaN,104651,NaN,...,NaN,MIMID,NaN,English,Microprocessors Microsyst,Article,Final,NaN,Scopus,2-s2.0-85139595586
2,Lilan C.; Yongsheng C.,"Lilan, Chen (57213686650); Yongsheng, Chen (65...",57213686650; 6507741439,Intelligent recommendation system based on dec...,2022,Frontiers in Computational Neuroscience,16,NaN,1048047,NaN,...,NaN,NaN,NaN,English,Front. Comput. Neurosci.,Article,Final,All Open Access; Gold Open Access; Green Open ...,Scopus,2-s2.0-85142172754
3,Min H.; Yang B.; Allen D.G.; Grandey A.A.; Liu M.,"Min, Hanyi (57194326042); Yang, Baojiang (5801...",57194326042; 58019611700; 57202914326; 6603322...,Wisdom from the crowd: Can recommender systems...,2022,Personnel Psychology,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,English,Pers. Psychol.,Article,Article in press,NaN,Scopus,2-s2.0-85144128378
4,Dahroug A.; Vlachidis A.; Liapis A.; Bikakis A...,"Dahroug, Ahmed (57205197369); Vlachidis, Andre...",57205197369; 36440453700; 53264261900; 1164101...,Using dates as contextual information for pers...,2021,Journal of Information Science,47,1,NaN,82,...,NaN,JISCD,NaN,English,J Inf Sci,Article,Final,All Open Access; Green Open Access,Scopus,2-s2.0-85073977929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,Han J.; Yamana H.,"Han, Jungkyu (56170953400); Yamana, Hayato (70...",56170953400; 7007103823,A study on individual mobility patterns based ...,2016,International Journal of Pervasive Computing a...,12,1,NaN,23,...,NaN,NaN,NaN,English,Int. J. Pervasive Comput. Commun.,Article,Final,NaN,Scopus,2-s2.0-84976339910
283,Kontiza K.; Loboda O.; Deladiennee L.; Castagn...,"Kontiza, K. (56241323100); Loboda, O. (5720398...",56241323100; 57203987002; 56344835300; 2308815...,A museum app to trigger users' reection,2018,CEUR Workshop Proceedings,2176,NaN,NaN,NaN,...,NaN,NaN,NaN,English,CEUR Workshop Proc.,Conference paper,Final,NaN,Scopus,2-s2.0-85052382796
284,Cardoso P.J.S.; Guerreiro P.; Monteiro J.; Rod...,"Cardoso, Pedro J. S. (35602693500); Guerreiro,...",35602693500; 57205268716; 35606413800; 3497329...,Applying an implicit recommender system in the...,2018,Lecture Notes in Computer Science (including s...,10908 LNCS,NaN,NaN,421,...,978-331992051-1,NaN,NaN,English,Lect. Notes Comput. Sci.,Conference paper,Final,NaN,Scopus,2-s2.0-85050585878
285,Herzog D.,"Herzog, Daniel (56435678200)",56435678200,Recommending a sequence of points of interest ...,2017,RecSys 2017 - Proceedings of the 11th ACM Conf...,NaN,NaN,NaN,402,...,978-145034652-8,NaN,NaN,English,RecSys - Proc. ACM Conf. Recomm. Syst.,Conference paper,Final,NaN,Scopus,2-s2.0-85030450668


### Checking for duplicate values

In [205]:
data.duplicated().sum()

0

since the value of duplicate values(rows) is zero is not necessary to do anything

### Cheking for null values


In [206]:
# Checking for missing values
data.isna().sum()

Authors                           39
Author full names                 39
Author(s) ID                      39
Title                              0
Year                               0
Source title                       0
Volume                           101
Issue                            215
Art. No.                         219
Page start                        90
Page end                          90
Page count                        54
Cited by                           0
DOI                               60
Link                               0
Affiliations                      41
Authors with affiliations         41
Abstract                           0
Author Keywords                   66
Index Keywords                    84
Molecular Sequence Numbers       287
Chemicals/CAS                    287
Tradenames                       287
Manufacturers                    287
Funding Details                  191
Funding Texts                    192
References                        41
C

### Remove useless columns in the context and columns with NaN values

In [207]:
columns_to_drop = ['Author full names', 'Volume', 'Issue', 'Art. No.', 'Page start', 'Page end', 'Page count', 'Cited by', 'DOI', 'Link', 'Affiliations', 'Authors with affiliations', 'Sponsors', 'Molecular Sequence Numbers', 'Chemicals/CAS', 'Tradenames', 'Manufacturers', 'ISBN', 'CODEN', 'PubMed ID']

data = data.drop(columns=columns_to_drop)
data.head(4)

,Authors,Author(s) ID,Title,Year,Source title,Abstract,Author Keywords,Index Keywords,Funding Details,Funding Texts,...,Conference location,Conference code,ISSN,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Open Access,Source,EID
0,Casillo M.; Conte D.; Lombardi M.; Santaniello...,23026997200; 22834021600; 57189595215; 5719787...,A Content-Based Recommender System for Hidden ...,2022,Lecture Notes in Networks and Systems,The enhancement of Italian artistic and cultur...,E-tourism; Recommender system; Singular value ...,NaN,NaN,NaN,...,"Virtual, Online",265119.0,23673370,English,Lect. Notes Networks Syst.,Conference paper,Final,NaN,Scopus,2-s2.0-85118993536
1,Rani C.J.; Devarakonda N.,57870952400; 54999915100,An effectual classical dance pose estimation a...,2022,Microprocessors and Microsystems,Automatic human action recognition remains an ...,Classification; Feature extraction; Indian cla...,Computer vision; Convolution; Convolutional ne...,NaN,NaN,...,NaN,NaN,01419331,English,Microprocessors Microsyst,Article,Final,NaN,Scopus,2-s2.0-85139595586
2,Lilan C.; Yongsheng C.,57213686650; 6507741439,Intelligent recommendation system based on dec...,2022,Frontiers in Computational Neuroscience,"How to recruit, test, and train the intelligen...",adaptive; classification algorithm; decision m...,Classification (of information); Efficiency; A...,Special Innovation Projects for Universities i...,This work was supported by the Major Projects ...,...,NaN,NaN,16625188,English,Front. Comput. Neurosci.,Article,Final,All Open Access; Gold Open Access; Green Open ...,Scopus,2-s2.0-85142172754
3,Min H.; Yang B.; Allen D.G.; Grandey A.A.; Liu M.,57194326042; 58019611700; 57202914326; 6603322...,Wisdom from the crowd: Can recommender systems...,2022,Personnel Psychology,Can algorithms that predict customer movie and...,Machine learning; Retention; Turnover; Work At...,NaN,NaN,NaN,...,NaN,NaN,00315826,English,Pers. Psychol.,Article,Article in press,NaN,Scopus,2-s2.0-85144128378


## **2. Preprocess the dataset**


In [208]:


# Function to perform text preprocessing
def preprocess_text(text):
    # Check if the input is a string
    if isinstance(text, str):
        # 1. Remove punctuation
        text = re.sub(r'[^\w\s]', '', text)

        # 2. Convert the text to lowercase
        text = text.lower()
        
        # 4. Tokenization (dividing the text to a list of words ex: 'a ana e linda ' -> ['a', 'ana', 'e' ,'linda'] )
        words = word_tokenize(text)

        
        # 5. Stopword Removal (remove very common words like 'the', 'a' etc.)
        stop_words = set(stopwords.words('english'))
        # In here he is removing the stop words from the text 
        words = [word for word in words if word.lower() not in stop_words]

    
        # Lemmatization
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word) for word in words]
        

        
        # 5. Stemming(reduse the words to their root form ex 'running' -> 'run')
        stemmer = PorterStemmer()
        words = [stemmer.stem(word) for word in words]
        
        # Join the processed words back into a sentence
        processed_text = ' '.join(words)
        
        return processed_text
    else:
        # Return an empty string for non-string values
        return ''
    

#create a list with all the text cloumns 
text_columns = data.select_dtypes(include='object').columns.tolist()

# Apply text preprocessing to specified text columns
for col in text_columns:
    data[col] = data[col].apply(preprocess_text)
# Drop duplicate rows based on the entire row
data = data.drop_duplicates()

# Display the updated DataFrame
data.head()


,Authors,Author(s) ID,Title,Year,Source title,Abstract,Author Keywords,Index Keywords,Funding Details,Funding Texts,...,Conference location,Conference code,ISSN,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Open Access,Source,EID
0,casillo cont lombardi santaniello troiano vale...,23026997200 22834021600 57189595215 5719787449...,contentbas recommend system hidden cultur heri...,2022,lectur note network system,enhanc italian artist cultur heritag big chall...,etour recommend system singular valu decomposit,,,,...,virtual onlin,265119.0,23673370,english,lect note network syst,confer paper,final,,scopu,2s2085118993536
1,rani cj devarakonda n,57870952400 54999915100,effectu classic danc pose estim classif system...,2022,microprocessor microsystem,automat human action recognit remain intric is...,classif featur extract indian classic danc pos...,comput vision convolut convolut neural network...,,,...,,NaN,01419331,english,microprocessor microsyst,articl,final,,scopu,2s2085139595586
2,lilan c yongsheng c,57213686650 6507741439,intellig recommend system base decis model arc...,2022,frontier comput neurosci,recruit test train intellig recommend system u...,adapt classif algorithm decis model intellig r...,classif inform effici adapt classif algorithm ...,special innov project univers guangdong 2021wt...,work support major project nation social scien...,...,,NaN,16625188,english,front comput neurosci,articl,final,open access gold open access green open access,scopu,2s2085142172754
3,min h yang b allen dg grandey aa liu,57194326042 58019611700 57202914326 6603322381...,wisdom crowd recommend system predict employe ...,2022,personnel psycholog,algorithm predict custom movi shop prefer also...,machin learn retent turnov work attitud,,,,...,,NaN,00315826,english,per psychol,articl,articl press,,scopu,2s2085144128378
4,dahroug vlachidi liapi bikaki lópeznor sacco p...,57205197369 36440453700 53264261900 1164101590...,use date contextu inform personalis cultur her...,2021,journal inform scienc,present semanticsbas mechan aim promot reflect...,annual commemor context awar cultur heritag hi...,recommend system semant annual commemor contex...,european union horizon 2020 research innov pro...,fund text 1 author disclos receipt follow fina...,...,,NaN,01655515,english,j inf sci,articl,final,open access green open access,scopu,2s2085073977929


## **3.Vectorizing Data: Bag-Of-Words ou 1 gram**

In [ ]:

# Assuming you have a column named 'Processed_Text' containing the preprocessed text
text_data = data['Processed_Text'].tolist()

# Initialize the CountVectorizer with 1-grams
vectorizer = CountVectorizer(ngram_range=(1, 1))

# Fit and transform the text data
X = vectorizer.fit_transform(text_data)

# Convert the result to a DataFrame for better visualization
vectorized_data = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Display the vectorized data
print(vectorized_data.head())